In [5]:
from requests_html import HTMLSession, HTML
from selenium import webdriver
import time
import re
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

In [2]:
#Kiểm tra xem dữ liệu có hợp pháp không
import urllib.robotparser
rp = urllib.robotparser.RobotFileParser()
rp.set_url('https://www.worldweatheronline.com/robots.txt')
rp.read()
rp.can_fetch('*', 'https://www.worldweatheronline.com/ho-chi-minh-city-weather-history/vn.aspx')

True

In [3]:
session = HTMLSession()

In [4]:
url = "https://www.worldweatheronline.com/ho-chi-minh-city-weather-history/vn.aspx"

In [5]:
#Tên file chứa data
filename='weather_data'

In [6]:
#tập hợp các loại thời tiết (để phân loại và chuyển về 1 số loại nhất định)
list_w = set()

In [9]:
def parse_data (url,filename,year):
    name = f"{filename}_{year}.csv"
    
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    browser = webdriver.Chrome(executable_path='/home/primedo/hcmus/DA/Datascience_2016-2/code/chromedriver', chrome_options=chrome_options)

    browser.get(url)
    for y in year:
        if ((y%4 == 0)|(y%400==0)):
            ferb_day=30
        else:
            ferb_day = 29
        #danh sách ngày trong tháng (để tìm kiếm ngày trong web)
        #thêm 0 để chỉ số bằng số tháng
        day = [[0],
                  range(1,32),
                  range(1,ferb_day),
                  range(1,32),
                  range(1,31),
                  range(1,32),
                  range(1,31),
                  range(1,32),
                  range(1,32),
                  range(1,31),
                  range(1,32),
                  range(1,31),
                  range(1,32)]    
        for m in range(12):
            m +=1
            if ((m==12)&(y==2019)):
                break;#Chỉ lấy đến tháng 11 năm 2019
            for d in day[m]:
                if (d<10):
                    d = f"0{d}"
                if (m<10):
                    m_str = f"0{m}"
                    text = f"{m_str}{d}{y}"
                else:
                    text = f"{m}{d}{y}"
                #Xác định vị trí của textbox và điền giá trị vào
                textbox = browser.find_element_by_xpath("//input[@id='ctl00_MainContentHolder_txtPastDate']")
                # textbox = browser.find_element("xpath", "//input[@id='ctl00_MainContentHolder_txtPastDate']")
                textbox.click()
                print(text)
                textbox.send_keys(text)
                #Bấm nút để lấy dữ liệu
                button = browser.find_element_by_xpath("//input[@id='ctl00_MainContentHolder_butShowPastWeather']")
                # button = browser.find_element("xpath", "//input[@id='ctl00_MainContentHolder_butShowPastWeather']")
                button.click()
                time.sleep(1)#Nghỉ 1 giây để trang có thời gian load
                
                r = HTML(html=browser.page_source)
                
                #kiểm tra xem page đã load đúng chưa
                #nếu sai thì tăng thời gian sleep
                print(r.find(".block_title",first=True).text)
                
                #lấy dữ liệu
                
                row = r.find(".tb_date",first=True)
                
                times =re.findall("\d{2}:\d{2}", row.text)
                
                row = r.find(".tb_temp",first=True)
                temp  = list(map(float,re.findall("(\d{1,2})",row.text)))
                
                row = r.find(".tb_feels",first=True)
                feels = list(map(float,re.findall("(\d{1,2})",row.text)))
                
                row = r.find(".tb_wind",first=True).text
                wind = list(map(float,re.findall("(\d{1,2}) km/h",row)))
                direction = re.findall(r"\n([A-Z]{1,3})",row)
                
                row = r.find(".tb_gust",first=True).text
                gust = list(map(float,re.findall("\d{1,2}",row)))
                
                row = r.find(".tb_cloud",first=True).text
                cloud = list(map(float,re.findall("\d{1,2}",row)))
                
                row = r.find(".tb_humidity",first=True).text
                humidity = list(map(float,re.findall("\d{1,2}",row)))
                
                row = r.find(".tb_precip",first=True).text
                precip = list(map(float,re.findall("(\d{1}.\d{1,3})",row)))
                
                row = r.find(".tb_pressure",first=True).text
                pressure = list(map(float,re.findall("\d{1,4}",row)))
                
                weather = []
                row = r.find(".tb_weather",first=True)
                att = row.find(".tb_item_img")
                for a in att:
                    weather += [a.attrs['alt']]
                    list_w.add(a.attrs['alt'])
                
                #ghi dữ liệu ra file csv
                with open(name,'w') as f:
                    f.write('time\tmonth\ttemperature\tfeelslike\twind\tdirection\tgust\tcloud\thumidity\tprecipitation\tpressure\tweather\n')
    
                with open(name,"a") as f:
                    for i in range(8):
                        f.write(f"{times[i]}\t{m}\t{temp[i]}\t{feels[i]}\t{wind[i]}\t{direction[i]}\t{gust[i]}\t{cloud[i]}\t{humidity[i]}\t{precip[i]}\t{pressure[i]}\t{weather[i]}\n")
                

Lưu dữ liệu ra 3 file riêng để nếu năm nào đó lấy bị lỗi chỉ cần crawl lại năm đó. Thỉnh thoảng driver chrome sẽ bị lỗi và trang không load được, sẽ có lỗi chính là các dòng ngày tháng in ra bị lặp lại

Ví dụ


12232017

Sat 23rd Dec, 2017

12242017

Sat 23rd Dec, 2017

12242017

Sat 23rd Dec, 2017


Nếu trong quá trình crawl gặp lỗi này thì chạy lại code và hạn chế tương tác với cửa số chrome bật lên, đồng thời, không được bấm nút minimize hoặc bất cư thao tác gì minimize cửa sổ đó (mở cửa sổ chrome khác để đè lên thì được)

In [10]:
parse_data(url,filename,[2018])

/tmp/ipykernel_1276573/1347045695.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path='/home/primedo/hcmus/DA/Datascience_2016-2/code/chromedriver', chrome_options=chrome_options)
/tmp/ipykernel_1276573/1347045695.py:6: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome(executable_path='/home/primedo/hcmus/DA/Datascience_2016-2/code/chromedriver', chrome_options=chrome_options)
/tmp/ipykernel_1276573/1347045695.py:42: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  textbox = browser.find_element_by_xpath("//input[@id='ctl00_MainContentHolder_txtPastDate']")
/tmp/ipykernel_1276573/1347045695.py:48: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  button = browser.find_element_by_xpath("//input[@id='ctl00_MainContentHolder_butShowP

01012018
Historical Weather on 01st January over the years
None


AttributeError: 'NoneType' object has no attribute 'text'

In [15]:
parse_data(url,filename,[2018])

01012018
Mon 01st Jan, 2018
01022018
Tue 02nd Jan, 2018
01032018
Wed 03rd Jan, 2018
01042018
Thu 04th Jan, 2018
01052018
Fri 05th Jan, 2018
01062018
Sat 06th Jan, 2018
01072018
Sun 07th Jan, 2018
01082018
Mon 08th Jan, 2018
01092018
Tue 09th Jan, 2018
01102018
Wed 10th Jan, 2018
01112018
Thu 11th Jan, 2018
01122018
Fri 12th Jan, 2018
01132018
Sat 13th Jan, 2018
01142018
Sun 14th Jan, 2018
01152018
Mon 15th Jan, 2018
01162018
Tue 16th Jan, 2018
01172018
Wed 17th Jan, 2018
01182018
Thu 18th Jan, 2018
01192018
Fri 19th Jan, 2018
01202018
Sat 20th Jan, 2018
01212018
Sun 21st Jan, 2018
01222018
Mon 22nd Jan, 2018
01232018
Tue 23rd Jan, 2018
01242018
Wed 24th Jan, 2018
01252018
Thu 25th Jan, 2018
01262018
Fri 26th Jan, 2018
01272018
Sat 27th Jan, 2018
01282018
Sun 28th Jan, 2018
01292018
Mon 29th Jan, 2018
01302018
Tue 30th Jan, 2018
01312018
Wed 31st Jan, 2018
02012018
Thu 01st Feb, 2018
02022018
Fri 02nd Feb, 2018
02032018
Sat 03rd Feb, 2018
02042018
Sun 04th Feb, 2018
02052018
Mon 05th Fe

10212018
Sun 21st Oct, 2018
10222018
Mon 22nd Oct, 2018
10232018
Tue 23rd Oct, 2018
10242018
Wed 24th Oct, 2018
10252018
Thu 25th Oct, 2018
10262018
Fri 26th Oct, 2018
10272018
Sat 27th Oct, 2018
10282018
Sun 28th Oct, 2018
10292018
Mon 29th Oct, 2018
10302018
Tue 30th Oct, 2018
10312018
Wed 31st Oct, 2018
11012018
Thu 01st Nov, 2018
11022018
Fri 02nd Nov, 2018
11032018
Sat 03rd Nov, 2018
11042018
Sun 04th Nov, 2018
11052018
Mon 05th Nov, 2018
11062018
Tue 06th Nov, 2018
11072018
Wed 07th Nov, 2018
11082018
Thu 08th Nov, 2018
11092018
Fri 09th Nov, 2018
11102018
Sat 10th Nov, 2018
11112018
Sun 11th Nov, 2018
11122018
Mon 12th Nov, 2018
11132018
Tue 13th Nov, 2018
11142018
Wed 14th Nov, 2018
11152018
Thu 15th Nov, 2018
11162018
Fri 16th Nov, 2018
11172018
Sat 17th Nov, 2018
11182018
Sun 18th Nov, 2018
11192018
Mon 19th Nov, 2018
11202018
Tue 20th Nov, 2018
11212018
Wed 21st Nov, 2018
11222018
Thu 22nd Nov, 2018
11232018
Fri 23rd Nov, 2018
11242018
Sat 24th Nov, 2018
11252018
Sun 25th No

In [16]:
parse_data(url,filename,[2019])#nam 2019 chỉ lấy đến hết tháng 11

01012019
Tue 01st Jan, 2019
01022019
Wed 02nd Jan, 2019
01032019
Thu 03rd Jan, 2019
01042019
Fri 04th Jan, 2019
01052019
Sat 05th Jan, 2019
01062019
Sun 06th Jan, 2019
01072019
Mon 07th Jan, 2019
01082019
Tue 08th Jan, 2019
01092019
Wed 09th Jan, 2019
01102019
Thu 10th Jan, 2019
01112019
Fri 11th Jan, 2019
01122019
Sat 12th Jan, 2019
01132019
Sun 13th Jan, 2019
01142019
Mon 14th Jan, 2019
01152019
Tue 15th Jan, 2019
01162019
Wed 16th Jan, 2019
01172019
Thu 17th Jan, 2019
01182019
Fri 18th Jan, 2019
01192019
Sat 19th Jan, 2019
01202019
Sun 20th Jan, 2019
01212019
Mon 21st Jan, 2019
01222019
Tue 22nd Jan, 2019
01232019
Wed 23rd Jan, 2019
01242019
Thu 24th Jan, 2019
01252019
Fri 25th Jan, 2019
01262019
Sat 26th Jan, 2019
01272019
Sun 27th Jan, 2019
01282019
Mon 28th Jan, 2019
01292019
Tue 29th Jan, 2019
01302019
Wed 30th Jan, 2019
01312019
Thu 31st Jan, 2019
02012019
Fri 01st Feb, 2019
02022019
Sat 02nd Feb, 2019
02032019
Sun 03rd Feb, 2019
02042019
Mon 04th Feb, 2019
02052019
Tue 05th Fe

10212019
Mon 21st Oct, 2019
10222019
Tue 22nd Oct, 2019
10232019
Wed 23rd Oct, 2019
10242019
Thu 24th Oct, 2019
10252019
Fri 25th Oct, 2019
10262019
Sat 26th Oct, 2019
10272019
Sun 27th Oct, 2019
10282019
Mon 28th Oct, 2019
10292019
Tue 29th Oct, 2019
10302019
Wed 30th Oct, 2019
10312019
Thu 31st Oct, 2019
11012019
Fri 01st Nov, 2019
11022019
Sat 02nd Nov, 2019
11032019
Sun 03rd Nov, 2019
11042019
Mon 04th Nov, 2019
11052019
Tue 05th Nov, 2019
11062019
Wed 06th Nov, 2019
11072019
Thu 07th Nov, 2019
11082019
Fri 08th Nov, 2019
11092019
Sat 09th Nov, 2019
11102019
Sun 10th Nov, 2019
11112019
Mon 11th Nov, 2019
11122019
Tue 12th Nov, 2019
11132019
Wed 13th Nov, 2019
11142019
Thu 14th Nov, 2019
11152019
Fri 15th Nov, 2019
11162019
Sat 16th Nov, 2019
11172019
Sun 17th Nov, 2019
11182019
Mon 18th Nov, 2019
11192019
Tue 19th Nov, 2019
11202019
Wed 20th Nov, 2019
11212019
Thu 21st Nov, 2019
11222019
Fri 22nd Nov, 2019
11232019
Sat 23rd Nov, 2019
11242019
Sun 24th Nov, 2019
11252019
Mon 25th No

In [ ]:
name = f"{filename}_{year[0]}"
name

In [3]:
url = "https://freemeteo.vn/thoi-tiet/ho-chi-minh-city/history/daily-history/?gid=1566083&date=2019-01-04&station=11437&language=vietnamese&country=vietnam&fbclid=IwAR2z7FPtgast1sYipfoEW6gsZMiX51hZOgafQ06Jid8tsatajFlAWPSQLV4"

In [289]:
chrome_options = Options()
chrome_options.add_argument("--headless")
browser = webdriver.Chrome(executable_path='/home/primedo/hcmus/DA/Datascience_2016-2/code/chromedriver', chrome_options=chrome_options)

browser.get(url)

/tmp/ipykernel_1369328/1173544627.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path='/home/primedo/hcmus/DA/Datascience_2016-2/code/chromedriver', chrome_options=chrome_options)
/tmp/ipykernel_1369328/1173544627.py:3: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome(executable_path='/home/primedo/hcmus/DA/Datascience_2016-2/code/chromedriver', chrome_options=chrome_options)


In [4]:
r = HTML(html=browser.page_source)

In [7]:
browser.find_element_by_class_name("cal").get_attribute("href")

/tmp/ipykernel_1369328/3337190325.py:1: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  browser.find_element_by_class_name("cal").get_attribute("href")


'javascript:void(0)'

In [58]:
browser.find_element_by_class_name("pagers").find_elements_by_tag_name("a")[-1].get_attribute("href")

/tmp/ipykernel_1289616/200667384.py:1: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  browser.find_element_by_class_name("pagers").find_elements_by_tag_name("a")[-1].get_attribute("href")
/tmp/ipykernel_1289616/200667384.py:1: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  browser.find_element_by_class_name("pagers").find_elements_by_tag_name("a")[-1].get_attribute("href")


'https://freemeteo.vn/thoi-tiet/ho-chi-minh-city/history/daily-history/?gid=1566083&date=2019-01-05&station=11437&language=vietnamese&country=vietnam'

In [57]:
import pandas as pd
pd.read_html(browser.page_source)[-4]

,Thời gian,Nhiệt độ,Nhiệt độ tương đối,Gió,Gió giật,Độ ẩm tương đối,Điểm sương,Áp suất,Biểu tượng,Mô tảChi tiết
0,00:00,27°C,29°C,90°17 Km/h,NaN,74%,22°C,"1013,0mb",NaN,"document.write(Icons.GetShortDescription(3, 'C..."
1,00:30,27°C,29°C,80°17 Km/h,NaN,74%,22°C,"1012,0mb",NaN,"document.write(Icons.GetShortDescription(3, 'C..."
2,01:00,27°C,29°C,70°9 Km/h,NaN,74%,22°C,"1013,0mb",NaN,"document.write(Icons.GetShortDescription(3, 'C..."
3,01:30,27°C,30°C,70°11 Km/h,NaN,79%,23°C,"1012,0mb",NaN,"document.write(Icons.GetShortDescription(3, 'C..."
4,02:00,26°C,28°C,70°9 Km/h,NaN,84%,23°C,"1012,0mb",NaN,"document.write(Icons.GetShortDescription(3, 'C..."
5,02:30,26°C,28°C,70°11 Km/h,NaN,84%,23°C,"1012,0mb",NaN,"document.write(Icons.GetShortDescription(3, 'C..."
6,03:00,26°C,28°C,60°11 Km/h,NaN,84%,23°C,"1012,0mb",NaN,"document.write(Icons.GetShortDescription(3, 'C..."
7,03:30,26°C,28°C,70°11 Km/h,NaN,84%,23°C,"1012,0mb",NaN,"document.write(Icons.GetShortDescription(3, 'C..."
8,04:00,25°C,26°C,60°11 Km/h,NaN,89%,23°C,"1012,0mb",NaN,"document.write(Icons.GetShortDescription(3, 'C..."
9,04:30,26°C,28°C,50°9 Km/h,NaN,84%,23°C,"1012,0mb",NaN,"document.write(Icons.GetShortDescription(3, 'C..."


In [42]:
pd.read_html(browser.page_source)[-4].drop(["Mô tảChi tiết"],axis=1)

,Thời gian,Nhiệt độ,Nhiệt độ tương đối,Gió,Gió giật,Độ ẩm tương đối,Điểm sương,Áp suất,Biểu tượng
0,00:00,28°C,30°C,100°19 Km/h,NaN,66%,21°C,"1013,0mb",NaN
1,00:30,27°C,29°C,100°19 Km/h,NaN,70%,21°C,"1013,0mb",NaN
2,01:00,27°C,29°C,110°11 Km/h,NaN,70%,21°C,"1012,0mb",NaN
3,01:30,27°C,29°C,120°11 Km/h,NaN,70%,21°C,"1012,0mb",NaN
4,02:00,27°C,29°C,120°9 Km/h,NaN,74%,22°C,"1012,0mb",NaN
5,02:30,26°C,28°C,110°9 Km/h,NaN,79%,22°C,"1012,0mb",NaN
6,03:00,26°C,28°C,120°7 Km/h,NaN,79%,22°C,"1012,0mb",NaN
7,03:30,26°C,28°C,110°7 Km/h,NaN,79%,22°C,"1011,0mb",NaN
8,04:00,26°C,28°C,110°9 Km/h,NaN,79%,22°C,"1012,0mb",NaN
9,04:30,26°C,28°C,110°7 Km/h,NaN,79%,22°C,"1012,0mb",NaN


In [290]:
url = "https://freemeteo.vn/thoi-tiet/vung-tau/history/daily-history/?gid=1562414&date=2022-07-20&station=11437&language=vietnamese&country=vietnam"
browser.get(url)

In [10]:
browser.find_element_by_class_name("drop-menu").get_attribute("class")

/tmp/ipykernel_1369328/2051059198.py:1: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  browser.find_element_by_class_name("drop-menu").get_attribute("class")


'drop-menu'

In [11]:
from bs4 import BeautifulSoup

In [45]:
soup = BeautifulSoup(browser.page_source, 'lxml')
data = []
table = soup.find_all('table')[1]
table_body = table.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele]) 

In [46]:
data

[[],
 ['Đỉnh  Fansipan',
  'Hôm nay | Ngày mai | 7 ngày',
  '2995m',
  '2112m',
  '1329m',
  '50 khách sạn'],
 ['Ta Liên',
  'Hôm nay | Ngày mai | 7 ngày',
  '2740m',
  '1963m',
  '1088m',
  '47 khách sạn'],
 ['Phu Si Lùng',
  'Hôm nay | Ngày mai | 7 ngày',
  '2706m',
  '2005m',
  '1234m',
  '0 khách sạn'],
 ['Núi Lang Cung',
  'Hôm nay | Ngày mai | 7 ngày',
  '2585m',
  '2003m',
  '1160m',
  '0 khách sạn'],
 ['Nam Kang Ho Tao',
  'Hôm nay | Ngày mai | 7 ngày',
  '2566m',
  '1637m',
  '1161m',
  '50 khách sạn'],
 ['Phu Xai Lai Leng',
  'Hôm nay | Ngày mai | 7 ngày',
  '2525m',
  '1582m',
  '1073m',
  '0 khách sạn'],
 ['Phu Sa Phin',
  'Hôm nay | Ngày mai | 7 ngày',
  '2493m',
  '1712m',
  '912m',
  '0 khách sạn'],
 ['Phu Kho Lường',
  'Hôm nay | Ngày mai | 7 ngày',
  '2455m',
  '1485m',
  '702m',
  '0 khách sạn'],
 ['Chư Yang Sin',
  'Hôm nay | Ngày mai | 7 ngày',
  '2405m',
  '1800m',
  '1033m',
  '2 khách sạn'],
 ['Phu Nằm Hạ',
  'Hôm nay | Ngày mai | 7 ngày',
  '2403m',
  '1739m',
 

In [52]:
parent = soup.find("body").find_all("ul")

In [53]:
parent

[<ul></ul>,
 <ul>
 <li><span class="left-shadow"></span><span class="left"></span><span class="menu-item" href="" style="width: 117.2px;"><span class="txt"><span class="ellipsis">Việt Nam</span></span></span><span class="right"></span><span class="right-shadow"></span><div class="drop-menu" style="left: -7px; width: 692px;">
 <div class="top"></div>
 <div class="contents"><span class="breadcrumb">Quốc gia</span><div class="countries-headers">
 <div class="col-title">Europe</div>
 <div class="col-title">America</div>
 <div class="col-title">Africa</div>
 <div class="col-title">Asia</div>
 <div class="col-title">Oceania</div>
 </div>
 <div class="table countries" style="height: 415px;">
 <div class="column first">
 <div class="country-list">
 <p class="index">A</p>
 <ul>
 <li><a href="/thoi-tiet/?language=vietnamese&amp;country=albania">Albania</a></li>
 <li><a href="/thoi-tiet/?language=vietnamese&amp;country=andorra">Andorra</a></li>
 </ul>
 <p class="index">Á</p>
 <ul>
 <li><a href="/

In [71]:
soup.find("div", attrs={'class': 'nomos-list'}).find_all("li")[0]

<li><a data-id="4743" href="" onclick="return false;">An Giang</a></li>

In [117]:
uls = browser.find_element_by_class_name("nomos-list").find_elements_by_tag_name("ul")

/tmp/ipykernel_1369328/189701825.py:1: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  uls = browser.find_element_by_class_name("nomos-list").find_elements_by_tag_name("ul")
/tmp/ipykernel_1369328/189701825.py:1: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  uls = browser.find_element_by_class_name("nomos-list").find_elements_by_tag_name("ul")


In [142]:
uls[0].find_elements_by_tag_name("a")[0].text

/tmp/ipykernel_1369328/3532467291.py:1: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  uls[0].find_elements_by_tag_name("a")[0].text


''

In [123]:
soup = BeautifulSoup(browser.page_source, 'lxml')
data = []
table = soup.find_all('table')[0]
table_body = table.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele]) 

In [124]:
data

[[],
 ['Hà Nội', 'Hôm nay | Ngày mai | 7 ngày', '26m', '1431270'],
 ['Hà Đông', 'Hôm nay | Ngày mai | 7 ngày', '27m', '50877'],
 ['Sơn Tây', 'Hôm nay | Ngày mai | 7 ngày', '23m', '40636'],
 ['Ai Mộ', 'Hôm nay | Ngày mai | 7 ngày', '19m', 'n/a'],
 ['An Khê', 'Hôm nay | Ngày mai | 7 ngày', '18m', 'n/a'],
 ['An Đà', 'Hôm nay | Ngày mai | 7 ngày', '18m', 'n/a'],
 ['Ang Phao', 'Hôm nay | Ngày mai | 7 ngày', '12m', 'n/a'],
 ['Ba Dương Thi', 'Hôm nay | Ngày mai | 7 ngày', '15m', 'n/a'],
 ['Ba Lăng', 'Hôm nay | Ngày mai | 7 ngày', '15m', 'n/a'],
 ['Ba Vì', 'Hôm nay | Ngày mai | 7 ngày', '336m', 'n/a'],
 ['Bai Sương', 'Hôm nay | Ngày mai | 7 ngày', '13m', 'n/a'],
 ['Bao Thon', 'Hôm nay | Ngày mai | 7 ngày', '9m', 'n/a'],
 ['Bá Dương Nội', 'Hôm nay | Ngày mai | 7 ngày', '17m', 'n/a'],
 ['Bách Kim', 'Hôm nay | Ngày mai | 7 ngày', '19m', 'n/a'],
 ['Bách Lộc', 'Hôm nay | Ngày mai | 7 ngày', '20m', 'n/a'],
 ['Bình Xá', 'Hôm nay | Ngày mai | 7 ngày', '19m', 'n/a'],
 ['Bạch Mai', 'Hôm nay | Ngày mai |

In [147]:
list_url = {
    "An Giang": "https://freemeteo.vn/thoi-tiet/long-xuyen/history/daily-history/?gid=1575627&station=11437&date=2022-07-22&language=vietnamese&country=vietnam",
    "BRVT": "https://freemeteo.vn/thoi-tiet/vung-tau/history/daily-history/?gid=1562414&station=11437&date=2022-07-22&language=vietnamese&country=vietnam",
    "Bạc Liêu": "https://freemeteo.vn/thoi-tiet/bac-lieu/history/daily-history/?gid=1591474&station=11437&date=2022-07-22&language=vietnamese&country=vietnam",
    "Bắc Giang": "https://freemeteo.vn/thoi-tiet/bac-giang/history/daily-history/?gid=1591527&station=11376&date=2022-07-22&language=vietnamese&country=vietnam",
    "Bắc Kạn": "https://freemeteo.vn/thoi-tiet/bac-kan/history/daily-history/?gid=1591538&station=11376&date=2022-07-22&language=vietnamese&country=vietnam",
    "Bắc Ninh": "https://freemeteo.vn/thoi-tiet/bac-ninh/history/daily-history/?gid=1591449&station=11376&date=2022-07-22&language=vietnamese&country=vietnam",
    "Bến Tre": "https://freemeteo.vn/thoi-tiet/ben-tre/history/daily-history/?gid=1587976&station=11437&date=2022-07-22&language=vietnamese&country=vietnam",
    "Bình Định": "https://freemeteo.vn/thoi-tiet/quy-nhon/history/daily-history/?gid=1568574&station=11397&date=2022-07-22&language=vietnamese&country=vietnam",
    "Bình Dương": "https://freemeteo.vn/thoi-tiet/thu-dau-mot/history/daily-history/?gid=1565022&station=11437&date=2022-07-22&language=vietnamese&country=vietnam",
    "Bình Phước": "https://freemeteo.vn/thoi-tiet/dong-xoai/history/daily-history/?gid=1582436&station=11437&date=2022-07-22&language=vietnamese&country=vietnam",
    "Bình Thuận": "https://freemeteo.vn/thoi-tiet/phan-thiet/history/daily-history/?gid=1571058&station=11437&date=2022-07-22&language=vietnamese&country=vietnam",
    "Cà Mau": "https://freemeteo.vn/thoi-tiet/ca-mau/history/daily-history/?gid=1586443&station=11437&date=2022-07-22&language=vietnamese&country=vietnam",
    "Cao Bằng": "https://freemeteo.vn/thoi-tiet/cao-bang/history/daily-history/?gid=1586185&station=11376&date=2022-07-22&language=vietnamese&country=vietnam",
    "Cần Thơ": "https://freemeteo.vn/thoi-tiet/can-tho/history/daily-history/?gid=1586203&station=11437&date=2022-07-22&language=vietnamese&country=vietnam",
    "Đà Nắng":"https://freemeteo.vn/thoi-tiet/da-nang/history/daily-history/?gid=1583992&station=11397&date=2022-07-22&language=vietnamese&country=vietnam",
    "Đak Lak": "https://freemeteo.vn/thoi-tiet/buon-ma-thuot/history/daily-history/?gid=1586896&station=11437&date=2022-07-22&language=vietnamese&country=vietnam",
    "Dak Nong": "https://freemeteo.vn/thoi-tiet/ban-ndoh/history/daily-history/?gid=1589388&station=11437&date=2022-07-22&language=vietnamese&country=vietnam",
    "Đồng Nai": "https://freemeteo.vn/thoi-tiet/bien-hoa/history/daily-history/?gid=1587923&station=11437&date=2022-07-22&language=vietnamese&country=vietnam",
    "Đồng Tháp": "https://freemeteo.vn/thoi-tiet/cao-lanh/history/daily-history/?gid=1586151&station=11437&date=2022-07-22&language=vietnamese&country=vietnam",
    "Gia Lai": "https://freemeteo.vn/thoi-tiet/pleiku/history/daily-history/?gid=1569684&station=11397&date=2022-07-22&language=vietnamese&country=vietnam",
    "Hà Giang": "https://freemeteo.vn/thoi-tiet/ha-giang/history/daily-history/?gid=1581349&station=11376&date=2022-07-22&language=vietnamese&country=vietnam",
    "Hà Nam": "https://freemeteo.vn/thoi-tiet/phu-ly/history/daily-history/?gid=1570449&station=11376&date=2022-07-22&language=vietnamese&country=vietnam",
    "Hà Nội": "https://freemeteo.vn/thoi-tiet/hanoi/history/daily-history/?gid=1581130&station=11376&date=2022-07-22&language=vietnamese&country=vietnam",
    "Hà Tây": "https://freemeteo.vn/thoi-tiet/ba-dinh/history/daily-history/?gid=7289846&station=11376&date=2022-07-22&language=vietnamese&country=vietnam",
    "Hà Tĩnh": "https://freemeteo.vn/thoi-tiet/ha-tinh/history/daily-history/?gid=1581047&station=11394&date=2022-07-22&language=vietnamese&country=vietnam",
    "Hải Dương": "https://freemeteo.vn/thoi-tiet/hai-duong/history/daily-history/?gid=1581326&station=11376&date=2022-07-22&language=vietnamese&country=vietnam",
    "Hải Phòng": "https://freemeteo.vn/thoi-tiet/haiphong/history/daily-history/?gid=1581298&station=11376&date=2022-07-22&language=vietnamese&country=vietnam",
    "Hậu Giang": "https://freemeteo.vn/thoi-tiet/ap-dong-binh-1/history/daily-history/?gid=1593537&station=11437&date=2022-07-22&language=vietnamese&country=vietnam",
    "Hòa Bình": "https://freemeteo.vn/thoi-tiet/hoa-binh/history/daily-history/?gid=1580830&station=11376&date=2022-07-22&language=vietnamese&country=vietnam",
    "Điện Biên": "https://freemeteo.vn/thoi-tiet/dien-bien-phu/history/daily-history/?gid=1583477&station=11376&date=2022-07-22&language=vietnamese&country=vietnam",
    "Hưng Yên": "https://freemeteo.vn/thoi-tiet/hung-yen/history/daily-history/?gid=1580142&station=11376&date=2022-07-22&language=vietnamese&country=vietnam",
    "Kiên Giang": "https://freemeteo.vn/thoi-tiet/rach-gia/history/daily-history/?gid=1568510&station=11437&date=2022-07-22&language=vietnamese&country=vietnam",
    "Kon Tum": "https://freemeteo.vn/thoi-tiet/kon-tum/history/daily-history/?gid=1578500&station=11397&date=2022-07-22&language=vietnamese&country=vietnam",
    "Khánh Hòa": "https://freemeteo.vn/thoi-tiet/nha-trang/history/daily-history/?gid=1572151&station=11437&date=2022-07-22&language=vietnamese&country=vietnam",
    "Lai Châu": "https://freemeteo.vn/thoi-tiet/a-me/history/daily-history/?gid=1594542&station=11376&date=2022-07-22&language=vietnamese&country=vietnam",
    "Lạng Sơn": "https://freemeteo.vn/thoi-tiet/lang-son/history/daily-history/?gid=1576633&station=11376&date=2022-07-22&language=vietnamese&country=vietnam",
    "Lào Cai": "https://freemeteo.vn/thoi-tiet/lao-cai/history/daily-history/?gid=1576303&station=11376&date=2022-07-22&language=vietnamese&country=vietnam",
    "Lâm Đồng": "https://freemeteo.vn/thoi-tiet/da-lat/history/daily-history/?gid=1584071&station=11437&date=2022-07-22&language=vietnamese&country=vietnam",
    "Long An": "https://freemeteo.vn/thoi-tiet/tan-an/history/daily-history/?gid=1567069&station=11437&date=2022-07-22&language=vietnamese&country=vietnam",
    "Nam Định": "https://freemeteo.vn/thoi-tiet/nam-dinh/history/daily-history/?gid=1573517&station=11376&date=2022-07-22&language=vietnamese&country=vietnam",
    "Ninh Bình": "https://freemeteo.vn/thoi-tiet/ninh-binh/history/daily-history/?gid=1571968&station=11376&date=2022-07-22&language=vietnamese&country=vietnam",
    "Ninh Thuận": "https://freemeteo.vn/thoi-tiet/phan-rang-thap-cham/history/daily-history/?gid=1571067&station=11437&date=2022-07-22&language=vietnamese&country=vietnam",
    "Nghệ An": "https://freemeteo.vn/thoi-tiet/vinh/history/daily-history/?gid=1562798&station=11376&date=2022-07-22&language=vietnamese&country=vietnam",
    "Phú Thọ": "https://freemeteo.vn/thoi-tiet/viet-tri/history/daily-history/?gid=1562820&station=11376&date=2022-07-22&language=vietnamese&country=vietnam",
    "Phú Yên": "https://freemeteo.vn/thoi-tiet/tuy-hoa/history/daily-history/?gid=1563281&station=11397&date=2022-07-22&language=vietnamese&country=vietnam",
    "Quảng Bình": "https://freemeteo.vn/thoi-tiet/dong-hoi/history/daily-history/?gid=1582886&station=11394&date=2022-07-22&language=vietnamese&country=vietnam",
    "Quảng Nam":"https://freemeteo.vn/thoi-tiet/tam-ky/history/daily-history/?gid=1567148&station=11397&date=2022-07-22&language=vietnamese&country=vietnam",
    "Quảng Ninh": "https://freemeteo.vn/thoi-tiet/ha-long/history/daily-history/?gid=1580410&station=11376&date=2022-07-22&language=vietnamese&country=vietnam",
    "Quảng Ngãi": "https://freemeteo.vn/thoi-tiet/quang-ngai/history/daily-history/?gid=1568770&station=11397&date=2022-07-22&language=vietnamese&country=vietnam",
    "Quảng Trị": "https://freemeteo.vn/thoi-tiet/dong-ha/history/daily-history/?gid=1582926&station=11394&date=2022-07-22&language=vietnamese&country=vietnam",
    "Sóc Trăng": "https://freemeteo.vn/thoi-tiet/soc-trang/history/daily-history/?gid=1567788&station=11437&date=2022-07-22&language=vietnamese&country=vietnam",
    "Sơn La": "https://freemeteo.vn/thoi-tiet/son-la/history/daily-history/?gid=1567681&station=11376&date=2022-07-22&language=vietnamese&country=vietnam",
    "Tây Ninh": "https://freemeteo.vn/thoi-tiet/tay-ninh/history/daily-history/?gid=1566559&station=11437&date=2022-07-22&language=vietnamese&country=vietnam",
    "Tiền Giang": "https://freemeteo.vn/thoi-tiet/my-tho/history/daily-history/?gid=1574023&station=11437&date=2022-07-22&language=vietnamese&country=vietnam",
    "Tuyên Quang": "https://freemeteo.vn/thoi-tiet/tuyen-quang/history/daily-history/?gid=1563287&station=11376&date=2022-07-22&language=vietnamese&country=vietnam",
    "Thái Bình": "https://freemeteo.vn/thoi-tiet/thai-binh/history/daily-history/?gid=1566346&station=11376&date=2022-07-22&language=vietnamese&country=vietnam",
    "Thái Nguyên": "https://freemeteo.vn/thoi-tiet/thai-nguyen/history/daily-history/?gid=1566319&station=11376&date=2022-07-22&language=vietnamese&country=vietnam",
    "Thanh Hóa": "https://freemeteo.vn/thoi-tiet/thanh-hoa/history/daily-history/?gid=1566166&station=11376&date=2022-07-22&language=vietnamese&country=vietnam",
    "Huế": "https://freemeteo.vn/thoi-tiet/hue/history/daily-history/?gid=1580240&station=11394&date=2022-07-22&language=vietnamese&country=vietnam",
    "Trà Vinh": "https://freemeteo.vn/thoi-tiet/tra-vinh/history/daily-history/?gid=1563926&station=11437&date=2022-07-22&language=vietnamese&country=vietnam",
    "Vĩnh Long": "https://freemeteo.vn/thoi-tiet/vinh-long/history/daily-history/?gid=1562693&station=11437&date=2022-07-22&language=vietnamese&country=vietnam",
    "Vĩnh Phúc": "https://freemeteo.vn/thoi-tiet/vinh-yen/history/daily-history/?gid=1562548&station=11376&date=2022-07-22&language=vietnamese&country=vietnam",
    "Yên Bái": "https://freemeteo.vn/thoi-tiet/yen-bai/history/daily-history/?gid=1560349&station=11376&date=2022-07-22&language=vietnamese&country=vietnam"
}

In [148]:
url = list_url["An Giang"]
chrome_options = Options()
chrome_options.add_argument("--headless")
browser = webdriver.Chrome(executable_path='/home/primedo/hcmus/DA/Datascience_2016-2/code/chromedriver', chrome_options=chrome_options)

browser.get(url)

/tmp/ipykernel_1369328/1785892022.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path='/home/primedo/hcmus/DA/Datascience_2016-2/code/chromedriver', chrome_options=chrome_options)
/tmp/ipykernel_1369328/1785892022.py:4: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome(executable_path='/home/primedo/hcmus/DA/Datascience_2016-2/code/chromedriver', chrome_options=chrome_options)


In [169]:
elems = browser.find_elements_by_xpath("//*[@class='drop-menu']")
elems[2]

/tmp/ipykernel_1369328/3959270667.py:1: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  elems = browser.find_elements_by_xpath("//*[@class='drop-menu']")


<selenium.webdriver.remote.webelement.WebElement (session="98fbe61f2f0670f6ff4a983fce0e089d", element="030d8dba-8eb4-4e81-aaca-2528f11ea8e1")>

In [181]:
elems[2].find_elements_by_class_name("contents")[0].find_elements_by_class_name("table")[0].find_elements_by_class_name("point-results-ui")[0].find_elements_by_tag_name("div")[1].find_elements_by_tag_name("div")[0].find_elements_by_tag_name("div")

/tmp/ipykernel_1369328/737003767.py:1: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  elems[2].find_elements_by_class_name("contents")[0].find_elements_by_class_name("table")[0].find_elements_by_class_name("point-results-ui")[0].find_elements_by_tag_name("div")[1].find_elements_by_tag_name("div")[0].find_elements_by_tag_name("div")


[]

In [235]:
browser.get(url)
browser.find_elements_by_xpath("//*[@class='point-results-ui g1']")[0].find_elements_by_class_name("nomos-list")[0].find_elements_by_tag_name("div")[1].find_elements_by_tag_name("li")[0].click()

/tmp/ipykernel_1369328/1321623923.py:2: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  browser.find_elements_by_xpath("//*[@class='point-results-ui g1']")[0].find_elements_by_class_name("nomos-list")[0].find_elements_by_tag_name("div")[1].find_elements_by_tag_name("li")[0].click()
/tmp/ipykernel_1369328/1321623923.py:2: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  browser.find_elements_by_xpath("//*[@class='point-results-ui g1']")[0].find_elements_by_class_name("nomos-list")[0].find_elements_by_tag_name("div")[1].find_elements_by_tag_name("li")[0].click()


ElementNotInteractableException: Message: element not interactable
  (Session info: headless chrome=96.0.4664.45)
Stacktrace:
#0 0x55bb8b687ee3 <unknown>
#1 0x55bb8b15549f <unknown>
#2 0x55bb8b18cbc6 <unknown>
#3 0x55bb8b1807b8 <unknown>
#4 0x55bb8b1a9272 <unknown>
#5 0x55bb8b180063 <unknown>
#6 0x55bb8b1a937e <unknown>
#7 0x55bb8b1bc3bc <unknown>
#8 0x55bb8b1a9163 <unknown>
#9 0x55bb8b17ebfc <unknown>
#10 0x55bb8b17fc05 <unknown>
#11 0x55bb8b6b9baa <unknown>
#12 0x55bb8b6cf651 <unknown>
#13 0x55bb8b6bab05 <unknown>
#14 0x55bb8b6d0a68 <unknown>
#15 0x55bb8b6af05f <unknown>
#16 0x55bb8b6eb818 <unknown>
#17 0x55bb8b6eb998 <unknown>
#18 0x55bb8b706eed <unknown>
#19 0x7fa00afd1609 <unknown>


In [165]:
# soup = BeautifulSoup(browser.page_source, 'html.parser')
# for link in soup.find_all('a'):
#     print(link.get('href'))

In [166]:
browser.find_element_by_css_selector("a[data-id ]>a")

/tmp/ipykernel_1369328/3622314284.py:1: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  browser.find_element_by_css_selector("a[data-id ]>a")


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"a[data-id ]>a"}
  (Session info: headless chrome=96.0.4664.45)
Stacktrace:
#0 0x55bb8b687ee3 <unknown>
#1 0x55bb8b155608 <unknown>
#2 0x55bb8b18baa1 <unknown>
#3 0x55bb8b18bc61 <unknown>
#4 0x55bb8b1be714 <unknown>
#5 0x55bb8b1a929d <unknown>
#6 0x55bb8b1bc3bc <unknown>
#7 0x55bb8b1a9163 <unknown>
#8 0x55bb8b17ebfc <unknown>
#9 0x55bb8b17fc05 <unknown>
#10 0x55bb8b6b9baa <unknown>
#11 0x55bb8b6cf651 <unknown>
#12 0x55bb8b6bab05 <unknown>
#13 0x55bb8b6d0a68 <unknown>
#14 0x55bb8b6af05f <unknown>
#15 0x55bb8b6eb818 <unknown>
#16 0x55bb8b6eb998 <unknown>
#17 0x55bb8b706eed <unknown>
#18 0x7fa00afd1609 <unknown>


In [236]:
import requests
url = "https://freemeteo.vn/thoi-tiet/cao-lanh/history/daily-history/?gid=1586151&station=11437&date=2020-01-01&language=vietnamese&country=vietnam"
r = requests.get(url)
print(r.text)

<!DOCTYPE html>
<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="vi">
<head>
<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
<meta http-equiv="Content-Language" content="vi" />
<title>Thời tiết Cao L&#227;nh – Lịch sử thời tiết h&#224;ng ng&#224;y| freemeteo.vn</title>
<meta name="KEYWORDS" content="Cao Lãnh lịch sử thời tiết hàng ngày, Cao Lãnh dữ liệu thời tiết lịch sử, Cao Lãnh thời tiết trong quá khứ">
<meta name="DESCRIPTION" content="Cao Lãnh thời tiết. Lịch sử thời tiết hàng ngày. Thời tiết hôm qua, tuần trước, tháng trước hoặc năm trước. Dữ liệu thời tiết chi tiết. freemeteo.vn">
<meta name="GENERATOR" content="Netvolution WCM" />
<base href="https://freemeteo.vn/" />
<link rel="stylesheet" type="text/css" href="//t1.fmet1.com/css/style.min9.css?v=101"  media="all" charset="utf-8"/>


<!--[if IE 8]><link rel="stylesheet" href="//t1.fmet1.com/css/ie8.css" type="text/css" media="screen, handheld, projection, tv" /><![endif]-->
<!--[if IE 7]><link rel="st

In [248]:
browser.get(url)
pd.read_html(browser.page_source)[-3]

,0,1,2,3,4,5
0,Dự báo hàng giờ,Hôm nay,Cuối tuần,7 ngày,Đồ thị,Trực tiếp qua vệ tinh


In [245]:
import pandas as pd
pd.read_html(r.text)[-1]

,0,1,2,3,4,5
0,Dự báo hàng giờ,Hôm nay,Cuối tuần,7 ngày,Đồ thị,Trực tiếp qua vệ tinh


In [2]:
chrome_options = Options()
chrome_options.add_argument("--headless")
browser = webdriver.Chrome(executable_path='/home/primedo/hcmus/DA/Datascience_2016-2/code/chromedriver', chrome_options=chrome_options)
browser.get("https://www.worldweatheronline.com/ho-chi-minh-city-weather-history/vn.aspx")

/tmp/ipykernel_1649425/633163425.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path='/home/primedo/hcmus/DA/Datascience_2016-2/code/chromedriver', chrome_options=chrome_options)
/tmp/ipykernel_1649425/633163425.py:3: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome(executable_path='/home/primedo/hcmus/DA/Datascience_2016-2/code/chromedriver', chrome_options=chrome_options)


In [7]:
soup = BeautifulSoup(browser.page_source, 'lxml')
data = []
table = soup.find_all('table')[0]
table_body = table.find('tbody')

rows = table_body.find_all('tr')
# for row in rows:
#     cols = row.find_all('td')
#     cols = [ele.text.strip() for ele in cols]
#     data.append([ele for ele in cols if ele]) # Get rid of empty values

In [8]:
row = rows[2]

In [9]:
cols = row.find_all('td')
# cols = [ele.text.strip() for ele in cols]
# cols

In [10]:
cols[-1]

<td class="days-details-row-item"><div class="days-arrow-box"><svg class="days-arrow days-arrow-white" style="transform: rotate(170.9deg);"><use href="/staticv150817/assets-202110/img/arrows.svg#hourly-arrow"></use></svg></div></td>

In [12]:
import re
re.findall("\(.*\)",cols[-1].find_all('svg')[0]["style"])[0].strip('()')

'170.9deg'

In [296]:
cols[1]

<td class="days-details-row-item"><img alt="Light rain" class="days-table-forecast-img" src="//cdn.worldweatheronline.com/images/weather/small/296_night_sm.png" title="Light rain"/></td>

In [287]:
cols[1].find_all('img')[0]['alt']

'Light rain'

In [256]:
data

[['Time',
  'Forecast',
  'Rain',
  'Rain %',
  'Cloud',
  'Pressure',
  'Wind',
  'Gust',
  'Dir'],
 [],
 ['00:0024 °c', '26 °c', '1.4mm', '0%', '81%', '1008mb', '8 km/h', '12 km/h'],
 ['03:0024 °c', '26 °c', '0.0mm', '0%', '74%', '1007mb', '9 km/h', '14 km/h'],
 ['06:0024 °c', '26 °c', '0.4mm', '0%', '65%', '1008mb', '7 km/h', '12 km/h'],
 ['09:0026 °c', '29 °c', '0.1mm', '0%', '61%', '1010mb', '10 km/h', '15 km/h'],
 ['12:0029 °c',
  '34 °c',
  '0.7mm',
  '0%',
  '100%',
  '1009mb',
  '16 km/h',
  '27 km/h'],
 ['15:0029 °c', '34 °c', '0.0mm', '0%', '82%', '1007mb', '16 km/h', '20 km/h'],
 ['18:0027 °c', '31 °c', '0.8mm', '0%', '79%', '1008mb', '6 km/h', '12 km/h'],
 ['21:0026 °c', '29 °c', '0.0mm', '0%', '47%', '1010mb', '3 km/h', '5 km/h']]